In [ ]:
import context
from models import NEURAL_FACE

import torch
from torch.utils.data import DataLoader

FORCE_CPU = False
import os
if FORCE_CPU:
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"

from core import utils_dataloader
from core import face_dataset

from glob import glob


import os.path as osp
import numpy as np
import torch
from psbody.mesh import Mesh
import pickle as pk
from utils import utils, writer, train_eval, mesh_sampling

In [ ]:
device = torch.device('cpu' if FORCE_CPU else 'cuda', 0)
print(device)

##### Paths:

In [ ]:
root = rf'{os.path.abspath(os.path.dirname(__file__))}/..'
data_path = rf'{root}/data'

In [ ]:
out_dir = rf'{data_path}/out_face_model'
logs_dir = out_dir + '/logs'
checkpoints_dir = out_dir + '/checkpoints'
parts_transforms_dir = out_dir + '/parts_transforms'
utils.makedirs(out_dir)
utils.makedirs(logs_dir)
utils.makedirs(checkpoints_dir)
utils.makedirs(parts_transforms_dir)
writer = writer.Writer(checkpoints_dir, logs_dir)

##### Dataset configs.

In [ ]:
TRAIN_MESH_COUNT = 150
TRAIN_VALID_SPLIT = 0.8

##### Building the mesh cache for faster load times.

In [ ]:
mesh_cache_path = rf'{out_dir}/mesh_cache.pkl'

In [ ]:
if not os.path.exists(mesh_cache_path):
    train_data, test_data, scaler = utils_dataloader.load_face_and_parts(data_path, TRAIN_MESH_COUNT)
    pk.dump((train_data, test_data, scaler), open(mesh_cache_path, 'wb'))

In [ ]:
train_data, test_data, scaler = pk.load(open(mesh_cache_path, 'rb'))

In [ ]:
print(len(train_data))
print(len(test_data))

In [ ]:
train_dataset = face_dataset.Dataset(train_data)
test_dataset = face_dataset.Dataset(test_data)

##### Save the scaler.

In [ ]:
pk.dump(scaler, open(rf'{out_dir}/face_scaler.pkl', 'wb'))

##### Load the parts info.

In [ ]:
part_verts = []
parts_names = []
parts_template = []
vert_map = np.loadtxt(rf'{data_path}/vert_map.csv', dtype=np.uint32)

for path in sorted(glob(rf'{data_path}/parts_info/*')):
    verts = np.loadtxt(path, dtype=np.uint32)
    for idx, vert in enumerate(verts):
        verts[idx] = vert_map[vert]
    part_verts.append(verts.tolist())
    part_name = os.path.basename(os.path.normpath(path)).split('.')[0]
    parts_names.append(part_name)
    parts_template.append(rf'{data_path}/mesh_data/parts/{part_name}_1.obj')
    

parts_num = len(parts_names)
print(parts_num)
print(parts_names)

##### Creating the down/up sampling matrices.

In [ ]:
ds_factors = [2, 2]
down_transform_list = []
down_edge_index_list = []

for part_name, template_fp in zip(parts_names, parts_template):
    transform_fp = rf'{parts_transforms_dir}/{part_name}.pkl'
    if not osp.exists(transform_fp):
        print('Generating transform matrices...')
        mesh = Mesh(filename=template_fp)
        
        _, A, D, U, F = mesh_sampling.generate_transform_matrices(mesh, ds_factors)
        tmp = {'face': F, 'adj': A, 'down_transform': D, 'up_transform': U}

        pk.dump(tmp, open(transform_fp, 'wb'))
        print('Done!')
        print('Transform matrices are saved in \'{}\''.format(transform_fp))
    else:
        tmp = pk.load(open(transform_fp, 'rb'), encoding='latin1')

    edge_index = [utils.to_edge_index(adj).to(device) for adj in tmp['adj']]
    down_transforms = [
        utils.to_sparse(down_transform).to(device)
        for down_transform in tmp['down_transform']
    ]

    down_edge_index_list.append(edge_index)
    down_transform_list.append(down_transforms)

In [ ]:
transform_fp = out_dir + '/face_transform.pkl'
template_fp = rf'{data_path}/mesh_data/faces/face (1).obj'

if not osp.exists(transform_fp):
    print('Generating transform matrices...')
    mesh = Mesh(filename=template_fp)

    _, A, D, U, F = mesh_sampling.generate_transform_matrices(mesh, ds_factors)
    tmp = {'face': F, 'adj': A, 'down_transform': D, 'up_transform': U}

    pk.dump(tmp, open(transform_fp, 'wb'))
    print('Done!')
    print('Transform matrices are saved in \'{}\''.format(transform_fp))
else:
    tmp = pk.load(open(transform_fp, 'rb'), encoding='latin1')

up_edge_index = [utils.to_edge_index(adj).to(device) for adj in tmp['adj']]
up_transforms = [
    utils.to_sparse(up_transform).to(device)
    for up_transform in tmp['up_transform']
]

##### Training

In [ ]:
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
K = 6
in_channels = 3
part_latent_size = 8
out_channels = [16, 32]

In [ ]:
epochs = 73

lr = 8e-4
lr_decay = 0.99

decay_step = 1
weight_decay = 0
beta = 1e-3  # 0.0055
ceta = 1e-4

In [ ]:
model = NEURAL_FACE(in_channels,
                    out_channels,
                    part_latent_size,
                    down_edge_index_list,
                    down_transform_list,
                    up_edge_index,
                    up_transforms,
                    K=K).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, decay_step, gamma=lr_decay)

##### For faster training, we don't save the model while training, and stop at epoch 73, which produces the best result.

In [ ]:
train_eval.run(model, train_loader, epochs, optimizer, scheduler, writer, device, part_verts,
                     part_latent_size, beta=beta, ceta=ceta, save_all=True, dont_save=True)

In [ ]:
writer.save_checkpoint(model, optimizer, scheduler, 73)